# Исследование уровня потребительской лояльности (NPS)

## Основная информация о проекте

**Заказчик исследования** — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. 

**Цель исследования** - определить текущий уровень потребительской лояльности или NPS (от англ. Net Promoter Score), среди клиентов из России.

**Задачи исследования:**
1. Определить, как участники распределены участники опроса по возрасту и полу;  
2. Исследовать, каких пользователей больше: новых или старых;
3. Выделить ТОП-5 городов по количеству респондентов;
4. Определите группы, которые наиболее и наименее лояльны к сервису;
5. Подсчитать общий NPS среди всех опрошенных и сделать выводы;
6. Описать портрет клиента, который является "сторонником" компании.

## Подключение к базе и выгрузка данных

In [2]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = '/Users/mimikhailova/Downloads/telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [4]:
query = """
SELECT u.user_id, 
       u.lt_day, 
       (CASE
           WHEN u.lt_day <= 365 THEN 'новый'
           ELSE 'старый'
       END) AS is_new, 
       u.age, 
       (CASE
           WHEN u.gender_segment = '0' THEN 'мужчина'
           WHEN u.gender_segment = '1' THEN 'женщина'
           ELSE 'другие'
       END) AS gender_segment,
       u.os_name, 
       u.cpe_type_name, 
       l.country, 
       l.city,
       SUBSTRING(age.title, 4) AS age_segment,
       SUBSTRING(traf.title, 4) AS traffic_segment,
       SUBSTRING(lt.title, 4) AS lifetime_segment,
       u.nps_score,
       (CASE
           WHEN u.nps_score >= 9 THEN 'сторонники'
           WHEN u.nps_score <= 6 THEN 'критики'
           ELSE 'нейтралы'
        END) AS nps_group
FROM user as u
LEFT JOIN location AS l on l.location_id=u.location_id
LEFT JOIN age_segment AS age on u.age_gr_id=age.age_gr_id
LEFT JOIN traffic_segment AS traf on u.tr_gr_id=traf.tr_gr_id
LEFT JOIN lifetime_segment AS lt on lt.lt_gr_id=u.lt_gr_id
"""

In [5]:
df = pd.read_sql(query, engine)

In [6]:
df.to_csv('telecomm_csi.csv', index=False)

## Дашборд

<a href="https://public.tableau.com/views/TelecommNPS_17219230195700/NPSDash?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link]">Ссылка на дашборд</a>

## Презентация и основные выводы по проекту

<a href="https://public.tableau.com/views/TelecommNPS_17219230195700/TELECOMM?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link">Ссылка на презентацию в Tableau</a>

### Основные выводы и наблюдения по проекту

#### Общая информация о клиентах

В опросе принимали участие **502493** пользователей. 

Большая часть опрошенных – **женщины** (54,22%). Также среди респондентов есть незначительное количество пользователей, не указавших информацию о своем поле – 0,26%. 

33,02% опрошенных находятся в возрастном диапазоне **от 35 до 44 лет**, 30,81% - **от 25 до 34**, причем категория 35-44 преобладает у обоих гендеров.

416393 пользователя (82,87%) являются **старыми**,причем более 50% клиентов  пользуются услугами **более 3-х лет**.

В топ-5 городов по количеству опрошенных пользователей вошли **Москва, Санкт-Петербург, Новосибирск, Екатеринбург, Казань**. 

#### Портрет "сторонника"

Сторонники представляют **наибольшую** долю среди всех опрошенных - 264 081 человек и 52,55%. Ими являются преимущественно **женщины** (56,57% опрошенных). По возрасту наибольшее количество сторонников находятся в категории **от 35 до 44 лет**. Сторонники являются **старыми** клиентами, которые пользуются услугами компании более года. Наибольшее количество сторонников проживает в **Москве, Санкт-Петербурге, Новосибирске, Казани и Екатеринбурге**. 

#### Показатель NPS

Общий показатель NPS равен **21,9%**. Данное значение сигнализирует о том, что несмотря на превалирующее количество сторонников, с бизнесом что-то **идет не так**. 

Один из вариантов решения проблемы – **проведение опроса среди пользователей** по поводу удовлетворения качеством оказываемых услуг, а также узнать о моментах, которые вызывают дискомфорт (интерфейс, работа службы поддержки и т.п). 